<a href="https://colab.research.google.com/github/SudeshRPatil20/Object-Detection-Models-Tutorial/blob/main/FCNs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn.functional as F

In [ ]:
import torchvision.models as model
vgg = model.vgg16(pretrained=True)

/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


100%|██████████| 528M/528M [00:06<00:00, 82.6MB/s]


In [ ]:
from torchsummary import summary


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

vgg = vgg.to(device)   # move model to GPU (or CPU if CUDA not available)
summary(vgg, (3, 224, 224), device=str(device))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
              ReLU-2         [-1, 64, 224, 224]               0
            Conv2d-3         [-1, 64, 224, 224]          36,928
              ReLU-4         [-1, 64, 224, 224]               0
         MaxPool2d-5         [-1, 64, 112, 112]               0
            Conv2d-6        [-1, 128, 112, 112]          73,856
              ReLU-7        [-1, 128, 112, 112]               0
            Conv2d-8        [-1, 128, 112, 112]         147,584
              ReLU-9        [-1, 128, 112, 112]               0
        MaxPool2d-10          [-1, 128, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]         295,168
             ReLU-12          [-1, 256, 56, 56]               0
           Conv2d-13          [-1, 256, 56, 56]         590,080
             ReLU-14          [-1, 256,

In [ ]:
import torch.nn as nn

class FCN8(nn.Module):
  def __init__(self, num_classes=2):
    super(FCN8, self).__init__()
    vgg=model.vgg16(pretrained=True)
    self.features=vgg.features

    self.pool3=self.features[:17]
    self.pool4=self.features[:24]
    self.pool5=self.features

    self.conv1x1_3=nn.conv2d(256, num_classes, kernel_size=1)
    self.conv1x1_4=nn.conv2d(512, num_classes, kernel_size=1)
    self.conv1x1_5=nn.conv2d(512, num_classes, kernel_size=1)

    self.upsample32=nn.ConvTranspose2d(num_classes, num_classes, kernel_size=4, stride=2, padding=1)
    self.upsample16=nn.ConvTranspose2d(num_classes, num_classes, kernel_size=4, stride=2, padding=1)
    self.upsample8=nn.ConvTranspose2d(num_classes, num_classes, kernel_size=16, stride=16, padding=4)


  def forward(self, x):
    pool3_out = self.pool3(x)
    pool4_out = self.pool4(x)
    pool5_out = self.pool5(x)

    pool3_out = self.conv1x1_3(pool3_out)
    pool4_out = self.conv1x1_4(pool4_out)
    pool5_out = self.conv1x1_5(pool5_out)

    x= self.upsample16(pool5_out) + pool4_out
    x= self.upsample32(x) + pool3_out
    x= self.upsample8(x)

    return x


In [ ]:
model = FCN8(num_classes=1000)
dummy_input = torch.randn(1, 3, 244, 244)
output = model(dummy_input)
print(output.shape)

/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


AttributeError: module 'torch.nn' has no attribute 'conv2d'